<a href="https://colab.research.google.com/github/zuckberj/jurisprudence-data-analysis/blob/master/jurisprudencia_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import requests
import aiohttp
import asyncio
import json

async def request_url(session, url, params):
    #proxies = {"http": "http://127.0.0.1:8080", "https": "http://127.0.0.1:8080"}
    #async with session.get(url+"/"+path,proxy="http://127.0.0.1:8080",ssl=ssl.SSLContext()) as response:
    try:
        headers = {'accept': 'application/json'}
        async with session.post(url,data=params,headers=headers) as response:
            if response.status != 200:
                return None
            return await response.text()
    except:
        return None

def query_builder():
  return {
    "query":{
        "function_score":{
            "query":{
                "bool":{
                    "filter":[
                        {
                            "query_string":{
                                "default_operator":"AND",
                                "fields":"",
                                "query":"",
                                "quote_analyzer":"standard_analyzer",
                                "quote_field_suffix":".standard",
                                "type":"cross_fields",
                                "fuzziness":"AUTO:4,7"
                            }
                        }
                    ]
                }
            }
        }
    },
    "_source":[
        "inteiro_teor_url"
    ],
    "aggs":{
        "base_agg":{
            "filters":{
                "filters":{
                    "acordaos":{
                        "match":{
                            "base":"acordaos"
                        }
                    },
                    "sumulas":{
                        "match":{
                            "base":"sumulas"
                        }
                    },
                    "decisoes":{
                        "match":{
                            "base":"decisoes"
                        }
                    },
                    "informativos":{
                        "match":{
                            "base":"informativos"
                        }
                    }
                }
            }
        }
    },
    "size":"",
    "from":""

}

In [ ]:
async def request_test_url():
    url_test = 'https://jurisprudencia.stf.jus.br/api/search/search'
    search_query = '*'
    page = 0
    size = 10
    params = '{"query":{"function_score":{"query":{"bool":{"filter":[{"query_string":{"default_operator":"AND","fields":,"query":"'+search_query+'","quote_analyzer":"standard_analyzer","quote_field_suffix":".standard","type":"cross_fields","fuzziness":"AUTO:4,7"}}]}}}},"_source":["inteiro_teor_url"],"aggs":{"base_agg":{"filters":{"filters":{"acordaos":{"match":{"base":"acordaos"}},"sumulas":{"match":{"base":"sumulas"}},"decisoes":{"match":{"base":"decisoes"}},"informativos":{"match":{"base":"informativos"}}}}}},"size":'+str(size)+',"from":'+str(page)+'}'
    params = {
        'query': {
            'function_score': {
                'query': {
                    'bool': {
                        'filter': [
                                   {
                                       'query_string': {
                                           'default_operator': 'AND',
                                           'fields': [
                                                      "acordao_ata^3",
                                                      "documental_acordao_mesmo_sentido_lista_texto",
                                                      "documental_doutrina_texto",
                                                      "documental_indexacao_texto",
                                                      "documental_jurisprudencia_citada_texto",
                                                      "documental_legislacao_citada_texto",
                                                      "documental_observacao_texto",
                                                      "documental_publicacao_lista_texto",
                                                      "documental_tese_tema_texto^3",
                                                      "documental_tese_texto^3",
                                                      "ementa_texto^3",
                                                      "ministro_facet",
                                                      "orgao_julgador",
                                                      "partes_lista_texto",
                                                      "procedencia_geografica_completo",
                                                      "processo_classe_processual_unificada_extenso",
                                                      "titulo^6",
                                                      "colac_numero",
                                                      "colac_pagina",
                                                      "decisao_texto^2",
                                                      "documental_decisao_mesmo_sentido_lista_texto",
                                                      "processo_precedente_texto",
                                                      "sumula_texto^3",
                                                      "conteudo_texto"
                                                      ],
                                              ''
                                       }
                                   }
                        ]
                    }
                }
            }
        }
    }
    async with aiohttp.ClientSession() as sessionTest:
        response=await RequestUrl(sessionTest,urlTest,params)
        print(response)

loop=asyncio.get_event_loop()
loop.create_task(RequestUrlTest())

<Task pending coro=<RequestUrlTest() running at <ipython-input-2-05b211564e61>:1>>

{"result":{"took":112,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":{"value":10000,"relation":"gte"},"max_score":0.0,"hits":[{"_index":"stf-idx20200519","_type":"_doc","_id":"colac26290","_score":0.0,"_source":{"inteiro_teor_url":"http://stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?numero=6093&classe=AI"}},{"_index":"stf-idx20200519","_type":"_doc","_id":"colac26289","_score":0.0,"_source":{"inteiro_teor_url":"http://stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?numero=6954&classe=AI"}},{"_index":"stf-idx20200519","_type":"_doc","_id":"colac26773","_score":0.0,"_source":{"inteiro_teor_url":"http://stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?numero=285&classe=MS"}},{"_index":"stf-idx20200519","_type":"_doc","_id":"colac28654","_score":0.0,"_source":{"inteiro_teor_url":"http://stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?numero=4081&classe=RvC"}},{"_index":"stf-idx20200519","_type":"_doc","_id":"colac29462","_score":

In [ ]:
async def ScrapSingleQuery(session,search_query,size,max_page=-1):
    url='https://jurisprudencia.stf.jus.br/api/search/search'
    index=0
    eof=False
    pdf_urls=[]
    while not eof and index!=max_page*size:
        params='{"query":{"function_score":{"query":{"bool":{"filter":[{"query_string":{"default_operator":"AND","fields":["acordao_ata^3","documental_acordao_mesmo_sentido_lista_texto","documental_doutrina_texto","documental_indexacao_texto","documental_jurisprudencia_citada_texto","documental_legislacao_citada_texto","documental_observacao_texto","documental_publicacao_lista_texto","documental_tese_tema_texto^3","documental_tese_texto^3","ementa_texto^3","ministro_facet","orgao_julgador","partes_lista_texto","procedencia_geografica_completo","processo_classe_processual_unificada_extenso","titulo^6","colac_numero","colac_pagina","decisao_texto^2","documental_decisao_mesmo_sentido_lista_texto","processo_precedente_texto","sumula_texto^3","conteudo_texto"],"query":"'+search_query+'","quote_analyzer":"standard_analyzer","quote_field_suffix":".standard","type":"cross_fields","fuzziness":"AUTO:4,7"}}]}}}},"_source":["inteiro_teor_url"],"aggs":{"base_agg":{"filters":{"filters":{"acordaos":{"match":{"base":"acordaos"}},"sumulas":{"match":{"base":"sumulas"}},"decisoes":{"match":{"base":"decisoes"}},"informativos":{"match":{"base":"informativos"}}}}}},"size":'+str(size)+',"from":'+str(index)+'}'
        response=await RequestUrl(session,url,params)
        results=json.loads(response)['result']['hits']['hits']
        pdf_urls=pdf_urls+[result['_source']['inteiro_teor_url'] for result in results if result['_source']!={}]
        if not results:
            eof=True
        index+=size
    return pdf_urls
        

In [ ]:
async def ScrapSingleQueryTest():
    search_query='Lula'
    size=150
    max_page=-1
    async with aiohttp.ClientSession() as session:
        result=await ScrapSingleQuery(session,search_query,size,max_page)
        print(len(result))
        print(result)

loop=asyncio.get_event_loop()
loop.create_task(ScrapSingleQueryTest())

<Task pending coro=<ScrapSingleQueryTest() running at <ipython-input-25-39cbc9e7b654>:1>>

63
['http://stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?numero=83154&classe=HC', 'http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=752235149', 'http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=752184399', 'http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=752783306', 'http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=750698504', 'http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=750698439', 'http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=749950185', 'http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=13534835', 'http://stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?numero=83337&classe=RE', 'http://stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?numero=524853&classe=AI-AgR', 'http://stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?numero=6031&classe=Rcl-AgR', 'http://stf.jus.br/portal/inteiroTeor/obter

In [ ]:
def Download(url):
    file_name = url.split('=')[1]+'.pdf'
    print(file_name)
    try:
        response = requests.get(url, stream=True)
        with open(file_name, "wb") as handle:
            for data in tqdm(response.iter_content()):
                handle.write(data)
        return True
    except Exception as e:
        print("Error while downloading. Trying Again. "+str(e))
        return False

In [ ]:
async def DownloadTest():
    urlTest='http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=3442694'
    response=Download(urlTest)
    print(response)

loop=asyncio.get_event_loop()
loop.create_task(DownloadTest())

<Task pending coro=<DownloadTest() running at <ipython-input-46-c0648cb6660d>:1>>

3442694.pdf


360141it [00:08, 44516.73it/s]


True


In [ ]:
def DownloadList(urls_):
    urls=set(urls_)
    for url in urls:
        result=Download(url)
        print(result)

In [ ]:
def DownloadListTest():
    urls=['http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=11480004','http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=11480004', 'http://www.stf.jus.br/portal/inteiroTeor/obterInteiroTeor.asp?idDocumento=3442694']
    result=DownloadList(urls)
    
DownloadListTest()

3442694.pdf


360141it [00:07, 45640.16it/s]


True
11480004.pdf


394772it [00:08, 48849.72it/s]

True
